In [ ]:
from pathlib import Path 

from PIL import Image
import torch 
from scr.model import load_llava, llava ask


device = "cuda" if torch.cuda.is_available() else "cpu"
model, processor = load_llava(device)

results = demo_llava_on_image(model, processor, "example.jpg")


In [ ]:
from transformers import LlavaForConditionalGeneration, AutoProcessor
from PIL import Image
import torch
from IPython.display import display


def load_llava(device: str | torch.device = "cpu"):
    model_name = "llava-hf/llava-1.5-7b-hf"

    if device == "cpu":
        torch_dtype = torch.float32
        device_map = None
    else:
        torch_dtype = torch.float16
        device_map = "auto"

    model = LlavaForConditionalGeneration.from_pretrained(
        model_name,
        torch_dtype=torch_dtype,
        device_map=device_map,
    )
    processor = AutoProcessor.from_pretrained(model_name)

    if device == "cpu":
        model.to(device)

    model.eval()
    return model, processor


def generate_llava_answer(
    model,
    processor,
    image: Image.Image,
    prompt: str,
    device: str | torch.device = "cpu",
    max_new_tokens: int = 128,
):
    """
    Аналог твоего generate_answer, но заточен под формат LLaVA-1.5:
    USER: <image>\n<prompt> ASSISTANT:
    """
    full_prompt = f"USER: <image>\n{prompt} ASSISTANT:"

    inputs = processor(
        text=full_prompt,
        images=image,
        return_tensors="pt",
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )

    # LLaVA возвращает только ответ ассистента
    answer = processor.batch_decode(output_ids, skip_special_tokens=True)[0]
    return answer.strip()


In [ ]:
def llava_caption(image_path: str):
    img = Image.open(image_path).convert("RGB")
    display(img.resize((384, 384)))

    prompt = (
        "Опиши изображение подробно: какие объекты видны, что они делают, "
        "как они расположены относительно друг друга, какой общий контекст сцены."
    )

    answer = generate_llava_answer(
        model=model,
        processor=processor,
        image=img,
        prompt=prompt,
        device=device,
        max_new_tokens=128,
    )

    print("\nCAPTION (LLaVA):")
    print(answer)


In [ ]:
def llava_qa(image_path: str, question: str, max_new_tokens: int = 64):
    img = Image.open(image_path).convert("RGB")
    display(img.resize((384, 384)))

    prompt = f"Вопрос: {question}\nОтветь максимально чётко и по существу."

    answer = generate_llava_answer(
        model=model,
        processor=processor,
        image=img,
        prompt=prompt,
        device=device,
        max_new_tokens=max_new_tokens,
    )
    print(f"\nQ: {question}")
    print("A:", answer)


In [ ]:
def llava_qa(image_path: str, question: str, max_new_tokens: int = 64):
    img = Image.open(image_path).convert("RGB")
    display(img.resize((384, 384)))

    prompt = f"Вопрос: {question}\nОтветь максимально чётко и по существу."

    answer = generate_llava_answer(
        model=model,
        processor=processor,
        image=img,
        prompt=prompt,
        device=device,
        max_new_tokens=max_new_tokens,
    )
    print(f"\nQ: {question}")
    print("A:", answer)


In [ ]:
llava_caption(image_paths["dog"])

llava_qa(image_paths["dog"], "Какая это порода собаки и по каким признакам ты так думаешь?")
llava_qa(image_paths["girl_cat"], "Где находится кошка и что делает человек?")
llava_qa(image_paths["cat"], "Это помещение внутри или снаружи? Обоснуй свой ответ.")


In [ ]:
def llava_reasoning_demo(image_path: str, max_new_tokens: int = 128):
    img = Image.open(image_path).convert("RGB")
    display(img.resize((384, 384)))

    tasks = {
        "rich_caption": (
            "Опиши сцену так, будто объясняешь её незрячему человеку. "
            "Не просто перечисли объекты, а расскажи, что они делают, "
            "как взаимодействуют и какой общий настрой у сцены."
        ),
        "before_after": (
            "Предположи, что могло происходить за несколько секунд до этого момента "
            "и что с высокой вероятностью произойдет через минуту. Объясни свою логику."
        ),
        "objects_and_layout": (
            "Перечисли основные объекты на изображении и опиши их примерное расположение "
            "словами (слева, справа, на переднем плане, на заднем плане, в центре и т.д.)."
        ),
        "counting_and_explanation": (
            "Сколько основных объектов одного типа ты видишь (например, людей, животных, машин)? "
            "Опиши, как ты считал и почему уверен в этом числе."
        ),
        "emotions_and_relations": (
            "Опиши вероятные эмоции людей или животных на фото и отношения между ними "
            "(доверие, игра, забота, осторожность и т.д.). Объясни, какие визуальные "
            "подсказки к этому приводят."
        ),
        "ocr_and_reasoning": (
            "Если на изображении есть текст, постарайся переписать его и объяснить, "
            "какую роль этот текст играет в контексте сцены."
        ),
        "story_mode": (
            "Придумай короткую историю (3-5 предложений), которая могла бы соответствовать "
            "этой фотографии. Используй детали с картинки и не выдумывай того, чего явно нет."
        ),
        "instagram_captions": (
            "Придумай три варианта подписи для Instagram к этому фото: "
            "1) шутливую, 2) тёплую/милую, 3) более философскую."
        ),
    }

    results = {}

    for name, prompt in tasks.items():
        print(f"\n================= {name} =================")
        answer = generate_llava_answer(
            model=model,
            processor=processor,
            image=img,
            prompt=prompt,
            device=device,
            max_new_tokens=max_new_tokens,
        )
        print(answer)
        results[name] = answer

    return results


In [ ]:
# Например, показать, что LLaVA «умнее» BLIP именно по рассуждению
llava_reasoning_demo(image_paths["girl_cat"])
llava_reasoning_demo(image_paths["dog"])


In [ ]:
def llava_chat_step(
    image: Image.Image,
    history: str,
    user_message: str,
    max_new_tokens: int = 128,
):
    """
    history — строка вида:
    'USER: ... ASSISTANT: ...</s>USER: ... ASSISTANT: ...'

    Мы просто добавляем ещё один USER/ASSISTANT и скармливаем LLaVA.
    """
    if history:
        prompt = (
            "История диалога:\n"
            f"{history}\n\n"
            "Сейчас продолжи диалог, учитывая картинку и контекст выше.\n\n"
            f"USER: {user_message}"
        )
    else:
        prompt = (
            "Ты — полезный ассистент, который рассуждает по изображению и ведёт диалог с пользователем.\n"
            f"USER: {user_message}"
        )

    # Оборачиваем всё в формат LLaVA-1.5
    full_prompt = (
        "USER: <image>\n"
        + prompt
        + " ASSISTANT:"
    )

    answer = generate_llava_answer(
        model=model,
        processor=processor,
        image=image,
        prompt=full_prompt,
        device=device,
        max_new_tokens=max_new_tokens,
    )

    # обновляем историю в формате LLaVA
    new_history = history + f"\nUSER: {user_message}\nASSISTANT: {answer}</s>"
    return answer, new_history


In [ ]:
img_chat = Image.open(image_paths["girl_cat"]).convert("RGB")
display(img_chat.resize((384, 384)))

history = ""

# Шаг 1
answer, history = llava_chat_step(
    img_chat,
    history=history,
    user_message="Опиши это фото кратко.",
)
print("Assistant:", answer)

# Шаг 2
answer, history = llava_chat_step(
    img_chat,
    history=history,
    user_message="Какие эмоции, по-твоему, испытывает человек?",
)
print("\nAssistant:", answer)

# Шаг 3
answer, history = llava_chat_step(
    img_chat,
    history=history,
    user_message="Придумай короткую подпись для Instagram для этого фото.",
)
print("\nAssistant:", answer)

print("\n--- FULL HISTORY ---")
print(history)
